In [1]:
# kaggle dependencies
########################
# pip install fastkaggle
# pip install wandb
# pip install polars
# pip install datasets
# pip install scikit-learn
# pip install evaluate

In [ ]:
import os

iskaggle = os.path.exists("/kaggle/input")
isremote = os.path.exists("/home/ubuntu")

In [ ]:
import os
import shutil
import fastkaggle
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from datasets import Dataset
import torch  # base
import torch.nn.functional as F
import json
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from pydantic import BaseModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset as TorchDataset
import evaluate
from torch.utils.data import DataLoader
from datetime import datetime
import subprocess
from sklearn.metrics import accuracy_score
from typing import List, Optional, Dict, Tuple, Union, Literal

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA (NVIDIA GPU)")
else:
    device = torch.device("cpu")
    print("Using CPU")

print(f"Device: {device}")

Using MPS (Apple Silicon GPU)
Device: mps


In [ ]:
if not iskaggle and not isremote:
    data_base_path = Path("./data")
    comp_name = "llm-classification-finetuning"
    datapath = data_base_path / comp_name
    if not os.path.exists(datapath) and not datapath.exists():
        install_path = fastkaggle.setup_comp(comp_name)
        shutil.move(install_path, datapath)

In [ ]:
if isremote:
    if os.getcwd() == "/home/ubuntu":
        os.chdir("./llm-classification-finetuning")

### Set up Kaggle/Local Env


In [ ]:
WANDB_PROJECT_NAME = "kaggle-llm-classification"


def setup_environment():
    """Detect environment and set up paths for both local and Kaggle"""

    if iskaggle:
        print("Running on Kaggle")

        INPUT_DIR = "/kaggle/input/llm-classification-finetuning"
        OUTPUT_DIR = "/kaggle/working"
        MODEL_DIR = "/kaggle/working/models"

        os.environ["WANDB_MODE"] = "disabled"
    else:
        print("💻 Running locally")

        INPUT_DIR = "./data/llm-classification-finetuning"
        OUTPUT_DIR = "./output"
        MODEL_DIR = "./models"

        os.environ["WANDB_PROJECT"] = WANDB_PROJECT_NAME
        os.environ["WANDB_LOG_MODEL"] = "false"
        os.environ["WANDB_WATCH"] = "false"
        # os.environ["WANDB_MODE"] = "disabled"

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(MODEL_DIR, exist_ok=True)
    # to kill warning when running in notebooks
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    return INPUT_DIR, OUTPUT_DIR, MODEL_DIR


INPUT_DIR, OUTPUT_DIR, MODEL_DIR = setup_environment()
print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Model directory: {MODEL_DIR}")

💻 Running locally
Input directory: ./data/llm-classification-finetuning
Output directory: ./output
Model directory: ./models


### Load Data


In [ ]:
try:
    df_train = pl.read_csv(f"{INPUT_DIR}/train.csv")
    df_test = pl.read_csv(f"{INPUT_DIR}/test.csv")
    df_sample = pl.read_csv(f"{INPUT_DIR}/sample_submission.csv")

    print("Data loaded successfully!")
    print(f"Train: {df_train.shape}")
    print(f"Test: {df_test.shape}")
    print(f"Sample submission: {df_sample.shape}")

except FileNotFoundError as e:
    print(f" Data file not found: {e}")
    print(f" Make sure data is in: {INPUT_DIR}")

    if os.path.exists(INPUT_DIR):
        files = os.listdir(INPUT_DIR)
        print(f"📁 Files in {INPUT_DIR}: {files}")

Data loaded successfully!
Train: (57477, 9)
Test: (3, 4)
Sample submission: (3, 4)


In [10]:
pl.Config.set_tbl_width_chars(300)
pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

In [11]:
df_train.sample(n=10)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
1572553183,"""claude-2.0""","""gpt-4-1106-preview""","""[""David Icke claims that Hitler was a Rothschild""]""","""[""I do not have enough evidence to definitively confirm or refute David Icke's claim that Hitler was a Rothschild. The Rothschild family is a wealthy Jewish banking dynasty, while Hitler was the dictator of Nazi Germany which perpetrated the Holocaust against Jewish people. Historians generally agree that Hitler's family background was lower-middle class Austrian. Claims that he had secret Rothschild ancestry tend to come from conspiracy theorists, not mainstream historians. Without clear documentation or evidence, the claim that Hitler was a Rothschild remains unproven speculation. Ultimately there are many open questions about Hitler's family history, but extraordinary claims require extraordinary evidence.""]""","""[""David Icke is a British conspiracy theorist known for his controversial claims and conspiracy theories, including those about global politics, health, and major world events. Among his many assertions is the claim that Adolf Hitler was connected to the Rothschild family, a wealthy Jewish banking dynasty often targeted by conspiracy theorists.\n\nIcke's theory suggests that Hitler's grandmother, Maria Anna Schicklgruber, worked for a Jewish family in Vienna, possibly the Rothschilds, and that Hitler's father, Alois Hitler, was the illegitimate child of a Rothschild. This claim is widely discredited and lacks credible evidence. Mainstream historical research shows that Alois Hitler's father was likely Johann Georg Hiedler.\n\nThe assertion that Hitler was a Rothschild is part of a broader set of conspiracy theories that allege various global elites, including the Rothschilds, are part of a secret cabal controlling world events. These theories often have anti-Semitic overtones, as they …",0,1,0
3620111529,"""gpt-4-0314""","""koala-13b""","""[""A CTF challenge provides a encryption service where you could connect and encrypt arbitrary messages or ask for the encrypted flag. The RSA key is regenerated at every connection using 512 bits primes and the RSA exponent is fixed to 17 (e=17) and the modulus N is unknown, regenerated at every connection). How can you retrieve the plaintext flag ? ""]""","""[""To retrieve the plaintext flag, you can use the following steps:\n\n1. Connect to the encryption service twice, to get two different connections with two different pairs of primes (p1, q1) and (p2, q2), but the same exponent e=17.\n\n2. Encrypt a known message 'M' on both connections to get the encrypted messages C1 and C2:\n C1 = M^17 mod (p1*q1)\n C2 = M^17 mod (p2*q2)\n\n3. Calculate the greatest common divisor (GCD) of (C1 - M^17) and (C2 - M^17). Since the exponent e=17 is the same, this GCD should give you one of the prime factors, let's say p1.\n\n GCD((C1 - M^17), (C2 - M^17)) = p1\n\n4. Now that you have p1, you can easily find q1 by dividing (p1*q1) by p1, as the modulus N = (p1*q1).\n\n q1 = (p1*q1) \/ p1\n\n5. Calculate the private exponent d using p1, q1, and e=17. To do this, first compute the Euler's totient function \u03a6(N):\n\n \u03a6(N) = (p1-1) * (q1-1)\n\nNext, find the modular inverse of e=17 with respect to \u03a6(N) using the extended Euclidean algo…","""[""It is not possible to retrieve the plaintext flag using only the information provided in the CTF challenge. The RSA encryption algorithm uses a public key and a private key, and the public key is known to everyone. The private key, which is used to decrypt the message, is kept secret and is not revealed to the public. Additionally, the modulus N is not known, so there is no way to determine the encryption key used to encrypt the plaintext flag. Therefore, it is not possible to retrieve the plaintext flag by using only the information provided in the CTF challenge.""]""",0,0,1
124797767,"""chatglm-6b""","

In [12]:
df_train["prompt"][0]

'["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]'

Try concatenating full conversations and full answer sets


In [ ]:
 
cols = ["prompt", "response_a", "response_b"]
if isinstance(df_train["prompt"][0], str):
    first_prompt_cell = df_train["prompt"][0]
    if "[" in first_prompt_cell and "]" in first_prompt_cell:
        df_train = df_train.with_columns([pl.col(col).str.json_decode() for col in cols])
        # explode
        df_train = df_train.explode(cols)
        # join
        # df_train = df_train.with_columns([pl.col(col).list.join(" ") for col in cols])

In [14]:
df_train["prompt"][0]

'Is it morally right to try to have a certain percentage of females on managerial positions?'

Remove any empty conversational turns:


In [ ]:
 
def null_clean(df: pl.DataFrame):
    df = df.drop_nulls(subset=["prompt", "response_a", "response_b"])
    return df

In [ ]:
 
df_train = null_clean(df_train)
df_test = null_clean(df_test)

In [17]:
df_train

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""Is it morally right to try to have a certain percentage of females on managerial positions?""","""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination. Here are some arguments in favor of and against such policies: **Arguments in favor:** 1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination. 2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole. 3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers. 4. **Role Modeling:** Increased v…","""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better decision-making. Critics may argue that positions should be based purely on merit, and that quotas could potentially lead to tokenism or unqualified candidates being promoted. Morality can be subjective and differs from person to person based on their personal beliefs, cultural background, and values. It's a topic with valid arguments on both sides, and it's up to each individual or organization to decide what they believe is right.""",1,0,0
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""OK, does pineapple belong on a pizza? Relax and give me fun answer.""","""Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle and pizza purists protest. Let's slice into the debate with a zest of fun: **Team Pineapple:** ""Absolutely, yes! Pineapple on pizza is like a beach party in your mouth. The sweet juiciness of pineapple chunks frolicking with savory ham or bacon creates a flavor wave that surfs across the cheesy ocean of deliciousness. It's the Mardi Gras of pizzas, where sweet meets savory in a jubilant jamboree!"" **Team No-Pineapple:** ""No way, not in a million pizzas! Pineapple is a fruit that should be sunbathing on a tropical fruit platter, not freeloading on a sacred slice of pizza. The thought of warm, melty cheese conspiring with pineapple's sugary mischief is enough to make Italian ancestors turn in their tomato patches. Keep the pineapple in the fruit salad where it…","""As an AI, I don't eat, so I don't have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Others firmly believe that the sweetness of the fruit has no place on a pizza. It's a pizza puzzle that might never be solved. So, whether pineapple belongs on a pizza or not, it truly depends on your taste buds!""",1,0,0
53567,"""koala-13b""","""gpt-4-0613""","""What is the difference between marriage license and marriage certificate?""","""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has ta

Format for `BERT` by concatenating the prompt and both responses with `[SEP]` tokens in between


In [18]:
("[CLS]" + df_train["prompt"] + "[SEP]" + df_train["response_a"] + "[SEP]" + df_train["response_b"])[0]

'[CLS]Is it morally right to try to have a certain percentage of females on managerial positions?[SEP]The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. **Equality of Opportunity:** Setting targets for female representation in management can

In [ ]:
 
def apply_bert_fmt(df: pl.DataFrame):
    df = df.with_columns(
        (
           "[CLS]" + pl.col("prompt") + "[SEP]" + pl.col("response_a") + "[SEP]" + pl.col("response_b")
        ).alias("text")
    )
    return df

Condense to a single target column and remap to 0,1,2 as possible dep vars


In [ ]:
 
df_train = df_train.with_columns(
    pl.when(pl.col("winner_model_a") == 1)
    .then(pl.lit(0))
    .when(pl.col("winner_model_b") == 1)
    .then(pl.lit(1))
    .when(pl.col("winner_tie") == 1)
    .then(pl.lit(2))
    .otherwise(pl.lit(None))
    .alias("label")
)

In [ ]:
 
df_train = apply_bert_fmt(df_train)
df_test = apply_bert_fmt(df_test)

Split the training data into training and validation sets


In [22]:
df_train.sample(n=10)

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,text
i64,str,str,str,str,str,i64,i64,i64,i32,str
1188170440,"""wizardlm-13b""","""alpaca-13b""","""Write a cute description of an apparel design. Do not specify what type of apparel it is for. The design is of a happy orange with the words “Squeeze the Day.” Do not start with the word “introducing.” Just make it sound summery and fun.""","""Are you ready to spread some sunshine and cheer? Look no further than our newest design, featuring a happy orange with the words ""Squeeze the Day""! This playful and summery design is sure to brighten up any outfit, and its positive message will inspire you to make the most out of every day. Whether you're lounging at the beach or running errands around town, this design will put a smile on your face and a bounce in your step. So why wait? Squeeze the day and make it yours!""","""Introducing the cutest summer apparel around! This orange tee is sure to make you smile with its bright color and cheerful slogan - ""Squeeze the Day"". Whether you're heading to the beach or just exploring your city, you'll be sure to stand out in this eye-catching design. So don't wait - grab your tee and start squeezing the day!""",1,0,0,0,"""[CLS]Write a cute description of an apparel design. Do not specify what type of apparel it is for. The design is of a happy orange with the words “Squeeze the Day.” Do not start with the word “introducing.” Just make it sound summery and fun.[SEP]Are you ready to spread some sunshine and cheer? Look no further than our newest design, featuring a happy orange with the words ""Squeeze the Day""! This playful and summery design is sure to brighten up any outfit, and its positive message will inspire you to make the most out of every day. Whether you're lounging at the beach or running errands around town, this design will put a smile on your face and a bounce in your step. So why wait? Squeeze the day and make it yours![SEP]Introducing the cutest summer apparel around! This orange tee is sure to make you smile with its bright color and cheerful slogan - ""Squeeze the Day"". Whether you're heading to the beach or just exploring your city, you'll be sure to stand out in this eye-catching design…"
1504517733,"""yi-34b-chat""","""chatglm3-6b""","""What if someone answered 4? What could be their reasoning?""","""If someone answered ""4"" for the number of murderers still in the room after one of them has been killed, their reasoning might be based on a misunderstanding of the scenario or a different interpretation of the word ""murderer."" Here are a few possible rationales for why someone might think there are four murderers in the room: 1. **Including the New Person as a Murderer**: The person might consider the person who enters the room and kills one of the other murderers to be a murderer as well, thus counting them as part of the group. 2. **Misunderstanding of the Scenario**: The person may not have fully processed the information given and may think that because the person who enters the room is also a murderer, there are now four murderers in total (the original three plus the new person). 3. **Death Does Not Change Status**: The person might believe that even though one murderer has been killed, they were still a murderer at the time of their death, and therefore should be counted. Th…","""If someone answered 4 murderers in the room, it is possible that their reasoning was based on the assumption that the person who entered the room and killed one of the other murderers is also a murderer, and they are counting the initial 3 murderers as well as the additional person who entered the room and committed a murder. In this case, the total number of murderers in the room would be 4, as the initial count of murderers remains unchanged, and the additional person who entered the room and committed a murder is now counted as a murderer as well. Again, it is important to note that the an

In [ ]:
 
if iskaggle:
    model_path = "/kaggle/input/bert-chatbot-preference/transformers/base-uncased-4epochs/1"
    print("Loading BERT from Kaggle model input...")

else:
    model_path = "bert-base-uncased"
    print("Loading BERT from Hugging Face...")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)
model.to(device)
model.device

Loading BERT from Hugging Face...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='mps', index=0)

In [ ]:
 
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train["text"], df_train["label"], test_size=0.1, random_state=42
)

### Load Dataset


In [ ]:
 
class LLMDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, labels=None, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    # encoding on the fly here due to issues with memory on kaggle
    # when pre-tokenizing
    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        item = {key: val.squeeze() for key, val in encoding.items()}

        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        return len(self.texts)

In [ ]:
 
train_dataset = LLMDataset(list(train_texts), tokenizer, list(train_labels))
val_dataset = LLMDataset(list(val_texts), tokenizer, list(val_labels))
test_dataset = LLMDataset(list(df_test["text"]), tokenizer)

### Inference


In [ ]:

text = "This is a test sentence"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs.to(device))
predictions = outputs.logits

In [ ]:
 
preds = F.softmax(predictions, dim=-1)

In [ ]:
 
all_probabilities = []
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

with torch.no_grad():
    for batch in test_dataloader:
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != "labels"}

        outputs = model(**inputs)
        probabilities = F.softmax(outputs.logits, dim=-1)
        all_probabilities.extend(probabilities.cpu().numpy())
final_probs = np.vstack(all_probabilities)

In [30]:
final_probs

array([[0.20722428, 0.3785305 , 0.41424525],
       [0.1915283 , 0.38792357, 0.42054814],
       [0.20198943, 0.385679  , 0.4123316 ]], dtype=float32)

In [31]:
final_probs[:, 0]

array([0.20722428, 0.1915283 , 0.20198943], dtype=float32)

In [ ]:
 
submission_df = df_test
submission_df = submission_df.with_columns(
    pl.lit(final_probs[:, 0]).alias("winner_model_a"),
    pl.lit(final_probs[:, 1]).alias("winner_model_b"),
    pl.lit(final_probs[:, 2]).alias("winner_tie"),
)
submission_df = submission_df[["id", "winner_model_a", "winner_model_b", "winner_tie"]]
submission_df

id,winner_model_a,winner_model_b,winner_tie
i64,f32,f32,f32
136060,0.207224,0.378531,0.414245
211333,0.191528,0.387924,0.420548
1233961,0.201989,0.385679,0.412332


In [ ]:
 
df_for_kaggle = submission_df.to_pandas()

In [ ]:
 
df_for_kaggle.to_csv("submission.csv", index=False)
df_for_kaggle.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.207224,0.378531,0.414245
1,211333,0.191528,0.387924,0.420548
2,1233961,0.201989,0.385679,0.412332


---


### Push Notebook to Kaggle


In [54]:
def push_notebook_cli():
    username = "peterbull"
    comp = "llm-classification-finetuning"
    notebook_file = "20250713_no_train.ipynb"
    kernel_id = f"{username}/llm-classification-bert-finetuning-no-train"
    
    # Create a unique metadata filename based on notebook
    metadata_file = f"kernel-metadata-{notebook_file.replace('.ipynb', '')}.json"
    
    # Default metadata (fallback if kernel doesn't exist remotely)
    default_metadata = {
        "id": kernel_id,
        "title": "LLM Classification BERT Finetuning No Train",
        "code_file": notebook_file,
        "language": "python",
        "kernel_type": "notebook",
        "is_private": True,
        "enable_gpu": True,
        "enable_internet": False,
        "dataset_sources": [],
        "competition_sources": [f"competitions/{comp}"],
        "kernel_sources": [],
        "model_sources": [
            "peterbull/bert-chatbot-preference"
        ],
    }

    # Try to pull existing metadata from Kaggle first
    if not os.path.exists(metadata_file):
        print(f"Metadata file {metadata_file} not found locally.")
        print("Attempting to pull remote metadata...")
        
        try:
            # Pull metadata from Kaggle
            result = subprocess.run(
                ["kaggle", "kernels", "metadata", kernel_id, "-p", "."],
                capture_output=True, text=True, timeout=60
            )
            
            if result.returncode == 0:
                print("✅ Successfully pulled remote metadata!")
                # Rename the downloaded file to our custom name
                if os.path.exists("kernel-metadata.json"):
                    os.rename("kernel-metadata.json", metadata_file)
                    print(f"📝 Renamed to {metadata_file}")
            else:
                print("⚠️ Kernel doesn't exist remotely, using default metadata")
                print("Creating new kernel with default settings...")
                with open(metadata_file, "w") as f:
                    json.dump(default_metadata, f, indent=2)
                    
        except subprocess.TimeoutExpired:
            print("⚠️ Metadata pull timed out, using default metadata")
            with open(metadata_file, "w") as f:
                json.dump(default_metadata, f, indent=2)
        except Exception as e:
            print(f"⚠️ Error pulling metadata: {e}")
            print("Using default metadata...")
            with open(metadata_file, "w") as f:
                json.dump(default_metadata, f, indent=2)
    else:
        print(f"✅ Using existing metadata file: {metadata_file}")

    # Verify notebook file exists
    if not os.path.exists(notebook_file):
        print("❌ Notebook file not found!")
        print("📁 Files in current directory:")
        for f in os.listdir("."):
            if f.endswith(".ipynb"):
                print(f"  {f}")
        return

    # Update the code_file in metadata to match current notebook
    try:
        with open(metadata_file, "r") as f:
            metadata = json.load(f)
        
        # Ensure code_file matches current notebook
        if metadata.get("code_file") != notebook_file:
            print(f"📝 Updating code_file from {metadata.get('code_file')} to {notebook_file}")
            metadata["code_file"] = notebook_file
            with open(metadata_file, "w") as f:
                json.dump(metadata, f, indent=2)
    except Exception as e:
        print(f"⚠️ Error reading metadata: {e}")
        return

    print(f"📤 Using metadata file: {metadata_file}")
    print("🚀 Pushing to Kaggle...")
    
    try:
        result = subprocess.run(
            ["kaggle", "kernels", "push", "-p", "."],
            capture_output=True, text=True, timeout=300
        )

        if result.returncode == 0:
            print("✅ Notebook pushed successfully!")
            print(result.stdout)
            print(f"🔗 View at: https://www.kaggle.com/code/{kernel_id}")
        else:
            print("❌ Error pushing notebook:")
            print(result.stderr)

    except subprocess.TimeoutExpired:
        print("⏰ Upload timed out after 5 minutes")
    except FileNotFoundError:
        print("❌ Kaggle CLI not found. Install with: pip install kaggle")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")


if not iskaggle:
    push_notebook_cli()


✅ Using existing metadata file: kernel-metadata-20250713_no_train.json
📤 Using metadata file: kernel-metadata-20250713_no_train.json
🚀 Pushing to Kaggle...
✅ Notebook pushed successfully!
The following are not valid tags and could not be added to the kernel: ['package']
Kernel version 13 successfully pushed.  Please check progress at https://www.kaggle.com/code/peterbull/llm-classification-bert-finetuning-no-train

🔗 View at: https://www.kaggle.com/code/peterbull/llm-classification-bert-finetuning-no-train
